# Proyecto Final de Ciencia de Datos

Este notebook implementa un pipeline completo de ciencia de datos que incluye:
1. **Extracción de datos**: Registro de dimensiones iniciales del dataset
2. **Limpieza de datos**: Documentación de valores nulos, duplicados e inconsistencias en tipos de datos
3. **Análisis Exploratorio de Datos (EDA)**: Distribuciones, frecuencias y estadísticos descriptivos
4. **Análisis Estadístico**: Planteamiento y evaluación de hipótesis
5. **Machine Learning**: Entrenamiento de cinco modelos de ML y evaluación de desempeño


## Importación de Librerías

In [ ]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Para análisis estadístico
from scipy import stats
from scipy.stats import normaltest, shapiro, chi2_contingency

# Para machine learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Librerías importadas exitosamente")

## 1. Extracción de Datos

En esta sección cargamos el dataset y registramos sus dimensiones iniciales.

In [ ]:
# Carga del dataset
# NOTA: Reemplaza 'dataset.csv' con la ruta a tu dataset
# Para este ejemplo, crearemos un dataset sintético para demostración

# Crear dataset sintético para demostración
np.random.seed(42)
n_samples = 1000

data = {
    'edad': np.random.randint(18, 80, n_samples),
    'ingresos': np.random.normal(50000, 15000, n_samples),
    'educacion': np.random.choice(['Secundaria', 'Universidad', 'Posgrado'], n_samples, p=[0.4, 0.4, 0.2]),
    'experiencia_anos': np.random.randint(0, 40, n_samples),
    'satisfaccion': np.random.randint(1, 6, n_samples),
    'categoria_cliente': np.random.choice(['A', 'B', 'C'], n_samples, p=[0.3, 0.5, 0.2])
}

# Introducir algunos valores nulos y duplicados para demostración
df = pd.DataFrame(data)
df.loc[np.random.choice(df.index, 50, replace=False), 'ingresos'] = np.nan
df.loc[np.random.choice(df.index, 30, replace=False), 'educacion'] = np.nan

# Añadir duplicados
duplicated_rows = df.sample(20)
df = pd.concat([df, duplicated_rows], ignore_index=True)

# Para cargar un dataset real, usa:
# df = pd.read_csv('ruta_a_tu_dataset.csv')

print("Dataset cargado exitosamente")

In [ ]:
# Registro de dimensiones iniciales del dataset
print("=== DIMENSIONES INICIALES DEL DATASET ===")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")
print(f"Tamaño total del dataset: {df.shape[0] * df.shape[1]} elementos")
print(f"Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n=== INFORMACIÓN GENERAL ===")
print(df.info())

print("\n=== PRIMERAS 5 FILAS ===")
display(df.head())

## 2. Limpieza de Datos

Documentamos y tratamos valores nulos, duplicados e inconsistencias en tipos de datos.

In [ ]:
# Análisis de valores nulos
print("=== ANÁLISIS DE VALORES NULOS ===")
null_counts = df.isnull().sum()
null_percentages = (df.isnull().sum() / len(df)) * 100

null_summary = pd.DataFrame({
    'Valores_Nulos': null_counts,
    'Porcentaje': null_percentages
})

print(null_summary[null_summary['Valores_Nulos'] > 0])

# Visualización de valores nulos
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), yticklabels=False, cbar=True, cmap='viridis')
plt.title('Patrón de Valores Nulos en el Dataset')
plt.show()

In [ ]:
# Análisis de duplicados
print("=== ANÁLISIS DE DUPLICADOS ===")
duplicated_count = df.duplicated().sum()
print(f"Número de filas duplicadas: {duplicated_count}")
print(f"Porcentaje de duplicados: {(duplicated_count / len(df)) * 100:.2f}%")

if duplicated_count > 0:
    print("\nPrimeras 5 filas duplicadas:")
    display(df[df.duplicated()].head())

In [ ]:
# Análisis de tipos de datos e inconsistencias
print("=== ANÁLISIS DE TIPOS DE DATOS ===")
print("\nTipos de datos actuales:")
print(df.dtypes)

print("\n=== DETECCIÓN DE INCONSISTENCIAS ===")

# Verificar valores únicos en variables categóricas
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    print(f"\nValores únicos en '{col}':")
    print(df[col].value_counts(dropna=False))

# Verificar rangos en variables numéricas
numerical_columns = df.select_dtypes(include=[np.number]).columns
print("\n=== RANGOS DE VARIABLES NUMÉRICAS ===")
for col in numerical_columns:
    print(f"\n{col}:")
    print(f"  Min: {df[col].min():.2f}")
    print(f"  Max: {df[col].max():.2f}")
    print(f"  Media: {df[col].mean():.2f}")
    print(f"  Mediana: {df[col].median():.2f}")

In [ ]:
# Aplicación de transformaciones necesarias
print("=== APLICANDO TRANSFORMACIONES ===")

# Crear una copia del dataset original
df_original = df.copy()
df_clean = df.copy()

# 1. Eliminar duplicados
print(f"Eliminando {df_clean.duplicated().sum()} filas duplicadas...")
df_clean = df_clean.drop_duplicates()

# 2. Tratar valores nulos
print("\nTratando valores nulos...")
# Para ingresos (numérica), usar la mediana
if 'ingresos' in df_clean.columns:
    median_income = df_clean['ingresos'].median()
    df_clean['ingresos'].fillna(median_income, inplace=True)
    print(f"  - Ingresos: {null_counts['ingresos']} valores nulos reemplazados con mediana ({median_income:.2f})")

# Para educación (categórica), usar la moda
if 'educacion' in df_clean.columns:
    mode_education = df_clean['educacion'].mode()[0]
    df_clean['educacion'].fillna(mode_education, inplace=True)
    print(f"  - Educación: {null_counts['educacion']} valores nulos reemplazados con moda ({mode_education})")

# 3. Verificar y corregir tipos de datos
print("\nVerificando tipos de datos...")
# Asegurar que las variables categóricas sean de tipo 'category'
categorical_cols = ['educacion', 'categoria_cliente']
for col in categorical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype('category')
        print(f"  - {col}: convertido a categórico")

# 4. Verificar rangos lógicos
print("\nVerificando rangos lógicos...")
# Edad debe estar entre 18 y 100
if 'edad' in df_clean.columns:
    outliers_edad = ((df_clean['edad'] < 18) | (df_clean['edad'] > 100)).sum()
    print(f"  - Edad: {outliers_edad} valores fuera del rango lógico (18-100)")

# Satisfacción debe estar entre 1 y 5
if 'satisfaccion' in df_clean.columns:
    outliers_sat = ((df_clean['satisfaccion'] < 1) | (df_clean['satisfaccion'] > 5)).sum()
    print(f"  - Satisfacción: {outliers_sat} valores fuera del rango lógico (1-5)")

print(f"\nDataset limpio - Filas: {df_clean.shape[0]}, Columnas: {df_clean.shape[1]}")
print(f"Filas eliminadas en el proceso: {df_original.shape[0] - df_clean.shape[0]}")

## 3. Análisis Exploratorio de Datos (EDA)

Exploramos las distribuciones, frecuencias y estadísticos descriptivos de nuestras variables.

In [ ]:
# Estadísticos descriptivos
print("=== ESTADÍSTICOS DESCRIPTIVOS ===")
print("\nVariables numéricas:")
display(df_clean.describe())

print("\nVariables categóricas:")
categorical_stats = df_clean.select_dtypes(include=['category', 'object']).describe()
if not categorical_stats.empty:
    display(categorical_stats)

In [ ]:
# Distribuciones de variables numéricas
print("=== DISTRIBUCIONES DE VARIABLES NUMÉRICAS ===")

numerical_cols = df_clean.select_dtypes(include=[np.number]).columns
n_cols = len(numerical_cols)
n_rows = (n_cols + 1) // 2

fig, axes = plt.subplots(n_rows, 2, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes]

for i, col in enumerate(numerical_cols):
    # Histograma
    axes[i].hist(df_clean[col].dropna(), bins=30, alpha=0.7, edgecolor='black')
    axes[i].set_title(f'Distribución de {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frecuencia')
    
    # Añadir línea de media
    mean_val = df_clean[col].mean()
    axes[i].axvline(mean_val, color='red', linestyle='--', 
                   label=f'Media: {mean_val:.2f}')
    axes[i].legend()

# Ocultar ejes vacíos
for i in range(len(numerical_cols), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Box plots para detectar outliers
print("=== DETECCIÓN DE OUTLIERS ===")

fig, axes = plt.subplots(n_rows, 2, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes]

for i, col in enumerate(numerical_cols):
    df_clean.boxplot(column=col, ax=axes[i])
    axes[i].set_title(f'Box Plot - {col}')
    axes[i].set_ylabel(col)

# Ocultar ejes vacíos
for i in range(len(numerical_cols), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Frecuencias de variables categóricas
print("=== FRECUENCIAS DE VARIABLES CATEGÓRICAS ===")

categorical_cols = df_clean.select_dtypes(include=['category', 'object']).columns
n_cat_cols = len(categorical_cols)

if n_cat_cols > 0:
    fig, axes = plt.subplots(1, n_cat_cols, figsize=(6*n_cat_cols, 6))
    if n_cat_cols == 1:
        axes = [axes]
    
    for i, col in enumerate(categorical_cols):
        # Contar frecuencias
        value_counts = df_clean[col].value_counts()
        
        # Gráfico de barras
        value_counts.plot(kind='bar', ax=axes[i], rot=45)
        axes[i].set_title(f'Frecuencias - {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frecuencia')
        
        # Añadir valores en las barras
        for j, v in enumerate(value_counts.values):
            axes[i].text(j, v + 0.1, str(v), ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar tablas de frecuencias
    for col in categorical_cols:
        print(f"\nTabla de frecuencias - {col}:")
        freq_table = df_clean[col].value_counts()
        freq_percentage = df_clean[col].value_counts(normalize=True) * 100
        
        freq_df = pd.DataFrame({
            'Frecuencia': freq_table,
            'Porcentaje': freq_percentage.round(2)
        })
        display(freq_df)
else:
    print("No hay variables categóricas en el dataset.")

In [ ]:
# Matriz de correlación
print("=== MATRIZ DE CORRELACIÓN ===")

# Calcular correlaciones solo para variables numéricas
numeric_df = df_clean.select_dtypes(include=[np.number])
correlation_matrix = numeric_df.corr()

# Crear heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('Matriz de Correlación entre Variables Numéricas')
plt.tight_layout()
plt.show()

# Identificar correlaciones fuertes
print("\nCorrelaciones más fuertes (|r| > 0.5):")
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
correlation_matrix_masked = correlation_matrix.mask(mask)

strong_correlations = []
for i in range(len(correlation_matrix_masked.columns)):
    for j in range(len(correlation_matrix_masked.columns)):
        val = correlation_matrix_masked.iloc[i, j]
        if pd.notna(val) and abs(val) > 0.5:
            strong_correlations.append((
                correlation_matrix_masked.columns[i],
                correlation_matrix_masked.columns[j],
                val
            ))

if strong_correlations:
    for var1, var2, corr in strong_correlations:
        print(f"  {var1} - {var2}: {corr:.3f}")
else:
    print("  No se encontraron correlaciones fuertes (|r| > 0.5)")

## 4. Análisis Estadístico

Planteamos y evaluamos hipótesis estadísticas sobre nuestros datos.

In [ ]:
print("=== ANÁLISIS ESTADÍSTICO - PLANTEAMIENTO DE HIPÓTESIS ===")

# Hipótesis 1: Los ingresos siguen una distribución normal
print("\n--- HIPÓTESIS 1: Normalidad de los Ingresos ---")
print("H0: Los ingresos siguen una distribución normal")
print("H1: Los ingresos NO siguen una distribución normal")

# Test de normalidad
ingresos_data = df_clean['ingresos'].dropna()

# Shapiro-Wilk test (para muestras < 5000)
if len(ingresos_data) < 5000:
    stat_shapiro, p_shapiro = shapiro(ingresos_data)
    print(f"\nShapiro-Wilk Test:")
    print(f"  Estadístico: {stat_shapiro:.4f}")
    print(f"  p-valor: {p_shapiro:.4f}")
    
    if p_shapiro < 0.05:
        print(f"  Resultado: RECHAZAMOS H0 (p < 0.05)")
        print(f"  Conclusión: Los ingresos NO siguen una distribución normal")
    else:
        print(f"  Resultado: NO rechazamos H0 (p ≥ 0.05)")
        print(f"  Conclusión: No hay evidencia suficiente para rechazar la normalidad")

# D'Agostino and Pearson's test
stat_dagostino, p_dagostino = normaltest(ingresos_data)
print(f"\nD'Agostino and Pearson's Test:")
print(f"  Estadístico: {stat_dagostino:.4f}")
print(f"  p-valor: {p_dagostino:.4f}")

if p_dagostino < 0.05:
    print(f"  Resultado: RECHAZAMOS H0 (p < 0.05)")
    print(f"  Conclusión: Los ingresos NO siguen una distribución normal")
else:
    print(f"  Resultado: NO rechazamos H0 (p ≥ 0.05)")
    print(f"  Conclusión: No hay evidencia suficiente para rechazar la normalidad")

# Visualización Q-Q plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Histograma con curva normal
ax1.hist(ingresos_data, bins=30, density=True, alpha=0.7, edgecolor='black')
xmin, xmax = ax1.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, ingresos_data.mean(), ingresos_data.std())
ax1.plot(x, p, 'r-', linewidth=2, label='Normal teórica')
ax1.set_title('Distribución de Ingresos vs Normal')
ax1.legend()

# Q-Q plot
stats.probplot(ingresos_data, dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot - Ingresos vs Normal')

plt.tight_layout()
plt.show()

In [ ]:
# Hipótesis 2: Existe diferencia significativa en ingresos por nivel educativo
print("\n--- HIPÓTESIS 2: Diferencia de Ingresos por Educación ---")
print("H0: No hay diferencia significativa en ingresos entre niveles educativos")
print("H1: Sí hay diferencia significativa en ingresos entre niveles educativos")

# Agrupar datos por educación
grupos_educacion = []
labels_educacion = []

for nivel in df_clean['educacion'].cat.categories:
    grupo_data = df_clean[df_clean['educacion'] == nivel]['ingresos'].dropna()
    if len(grupo_data) > 0:
        grupos_educacion.append(grupo_data)
        labels_educacion.append(nivel)

# ANOVA de una vía
if len(grupos_educacion) > 2:
    stat_anova, p_anova = stats.f_oneway(*grupos_educacion)
    print(f"\nANOVA de una vía:")
    print(f"  Estadístico F: {stat_anova:.4f}")
    print(f"  p-valor: {p_anova:.4f}")
    
    if p_anova < 0.05:
        print(f"  Resultado: RECHAZAMOS H0 (p < 0.05)")
        print(f"  Conclusión: SÍ hay diferencia significativa en ingresos por educación")
    else:
        print(f"  Resultado: NO rechazamos H0 (p ≥ 0.05)")
        print(f"  Conclusión: No hay evidencia suficiente de diferencias significativas")

# Estadísticas descriptivas por grupo
print("\nEstadísticas por nivel educativo:")
for i, (grupo, label) in enumerate(zip(grupos_educacion, labels_educacion)):
    print(f"  {label}: Media = {grupo.mean():.2f}, Mediana = {grupo.median():.2f}, n = {len(grupo)}")

# Visualización
plt.figure(figsize=(10, 6))
df_clean.boxplot(column='ingresos', by='educacion', ax=plt.gca())
plt.title('Distribución de Ingresos por Nivel Educativo')
plt.suptitle('')  # Remover título automático
plt.xlabel('Nivel Educativo')
plt.ylabel('Ingresos')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Hipótesis 3: Existe asociación entre nivel educativo y categoría de cliente
print("\n--- HIPÓTESIS 3: Asociación Educación-Categoría Cliente ---")
print("H0: No hay asociación entre nivel educativo y categoría de cliente")
print("H1: Sí hay asociación entre nivel educativo y categoría de cliente")

# Crear tabla de contingencia
contingency_table = pd.crosstab(df_clean['educacion'], df_clean['categoria_cliente'])
print("\nTabla de Contingencia:")
display(contingency_table)

# Test Chi-cuadrado
chi2_stat, p_chi2, dof, expected = chi2_contingency(contingency_table)

print(f"\nTest Chi-cuadrado de independencia:")
print(f"  Estadístico Chi-cuadrado: {chi2_stat:.4f}")
print(f"  p-valor: {p_chi2:.4f}")
print(f"  Grados de libertad: {dof}")

if p_chi2 < 0.05:
    print(f"  Resultado: RECHAZAMOS H0 (p < 0.05)")
    print(f"  Conclusión: SÍ hay asociación significativa entre educación y categoría")
else:
    print(f"  Resultado: NO rechazamos H0 (p ≥ 0.05)")
    print(f"  Conclusión: No hay evidencia suficiente de asociación")

# Visualización de la tabla de contingencia
plt.figure(figsize=(8, 6))
sns.heatmap(contingency_table, annot=True, fmt='d', cmap='Blues')
plt.title('Tabla de Contingencia: Educación vs Categoría Cliente')
plt.ylabel('Nivel Educativo')
plt.xlabel('Categoría Cliente')
plt.tight_layout()
plt.show()

# Mostrar frecuencias esperadas
print("\nFrecuencias esperadas bajo H0:")
expected_df = pd.DataFrame(expected, 
                          index=contingency_table.index, 
                          columns=contingency_table.columns)
display(expected_df.round(2))

## 5. Machine Learning

Entrenamos cinco modelos de Machine Learning y evaluamos su desempeño.

In [ ]:
print("=== PREPARACIÓN DE DATOS PARA MACHINE LEARNING ===")

# Definir variable objetivo (para este ejemplo, predeciremos la categoría de cliente)
target_column = 'categoria_cliente'
feature_columns = [col for col in df_clean.columns if col != target_column]

print(f"Variable objetivo: {target_column}")
print(f"Variables predictoras: {feature_columns}")

# Preparar características (X) y variable objetivo (y)
X = df_clean[feature_columns].copy()
y = df_clean[target_column].copy()

# Codificar variables categóricas
label_encoders = {}
categorical_features = X.select_dtypes(include=['object', 'category']).columns

for col in categorical_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
    print(f"Variable '{col}' codificada")

# Codificar variable objetivo
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)

print(f"\nClases en variable objetivo: {target_encoder.classes_}")
print(f"Distribución de clases:")
unique, counts = np.unique(y_encoded, return_counts=True)
for i, (clase, count) in enumerate(zip(target_encoder.classes_, counts)):
    print(f"  {clase}: {count} ({count/len(y_encoded)*100:.1f}%)")

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\nTamaño del conjunto de entrenamiento: {X_train.shape[0]}")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]}")

# Normalizar características numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nDatos preparados exitosamente para ML")

In [ ]:
# Definir los cinco modelos de Machine Learning
print("=== ENTRENAMIENTO DE MODELOS DE MACHINE LEARNING ===")

models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'SVM': SVC(random_state=42, probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100)
}

# Diccionario para almacenar resultados
results = {}

print("Entrenando modelos...\n")

for name, model in models.items():
    print(f"--- {name} ---")
    
    # Decidir si usar datos escalados o no
    if name in ['Logistic Regression', 'SVM', 'K-Nearest Neighbors']:
        X_train_use = X_train_scaled
        X_test_use = X_test_scaled
        print("  Usando datos escalados")
    else:
        X_train_use = X_train
        X_test_use = X_test
        print("  Usando datos originales")
    
    # Entrenar modelo
    model.fit(X_train_use, y_train)
    
    # Predicciones
    y_pred = model.predict(X_test_use)
    y_pred_proba = model.predict_proba(X_test_use)
    
    # Métricas de evaluación
    accuracy = accuracy_score(y_test, y_pred)
    
    # Validación cruzada
    cv_scores = cross_val_score(model, X_train_use, y_train, cv=5, scoring='accuracy')
    
    # AUC-ROC (para problemas multiclase, usar macro average)
    try:
        auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')
    except:
        auc_score = 'N/A'
    
    # Almacenar resultados
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'auc_score': auc_score,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")
    print(f"  AUC Score: {auc_score if auc_score != 'N/A' else 'N/A'}")
    print()

print("Entrenamiento completado")

In [ ]:
# Evaluación comparativa de modelos
print("=== EVALUACIÓN COMPARATIVA DE MODELOS ===")

# Crear tabla de comparación
comparison_data = []
for name, result in results.items():
    comparison_data.append({
        'Modelo': name,
        'Accuracy': f"{result['accuracy']:.4f}",
        'CV Mean': f"{result['cv_mean']:.4f}",
        'CV Std': f"{result['cv_std']:.4f}",
        'AUC Score': f"{result['auc_score']:.4f}" if result['auc_score'] != 'N/A' else 'N/A'
    })

comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)

# Identificar el mejor modelo
best_model_name = max(results.keys(), key=lambda x: results[x]['cv_mean'])
print(f"\nMejor modelo basado en CV Score: {best_model_name}")
print(f"CV Score: {results[best_model_name]['cv_mean']:.4f}")

# Visualización comparativa
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras - Accuracy
models_names = list(results.keys())
accuracies = [results[name]['accuracy'] for name in models_names]
cv_means = [results[name]['cv_mean'] for name in models_names]

x = np.arange(len(models_names))
width = 0.35

ax1.bar(x - width/2, accuracies, width, label='Test Accuracy', alpha=0.8)
ax1.bar(x + width/2, cv_means, width, label='CV Mean', alpha=0.8)
ax1.set_xlabel('Modelos')
ax1.set_ylabel('Score')
ax1.set_title('Comparación de Accuracy y CV Score')
ax1.set_xticks(x)
ax1.set_xticklabels(models_names, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Box plot de CV scores
cv_scores_all = []
model_labels = []
for name, model in models.items():
    if name in ['Logistic Regression', 'SVM', 'K-Nearest Neighbors']:
        X_use = X_train_scaled
    else:
        X_use = X_train
    
    cv_scores = cross_val_score(model, X_use, y_train, cv=5, scoring='accuracy')
    cv_scores_all.append(cv_scores)
    model_labels.append(name)

ax2.boxplot(cv_scores_all, labels=model_labels)
ax2.set_title('Distribución de CV Scores')
ax2.set_ylabel('CV Score')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis detallado del mejor modelo
print(f"=== ANÁLISIS DETALLADO DEL MEJOR MODELO: {best_model_name} ===")

best_model = results[best_model_name]['model']
best_y_pred = results[best_model_name]['y_pred']

# Reporte de clasificación
print("\nReporte de Clasificación:")
class_names = target_encoder.classes_
print(classification_report(y_test, best_y_pred, target_names=class_names))

# Matriz de confusión
cm = confusion_matrix(y_test, best_y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title(f'Matriz de Confusión - {best_model_name}')
plt.ylabel('Valor Real')
plt.xlabel('Predicción')
plt.tight_layout()
plt.show()

# Importancia de características (si está disponible)
if hasattr(best_model, 'feature_importances_'):
    print("\nImportancia de Características:")
    feature_importance = pd.DataFrame({
        'Feature': feature_columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    display(feature_importance)
    
    # Visualización de importancia
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance.head(10), x='Importance', y='Feature')
    plt.title(f'Top 10 Características Más Importantes - {best_model_name}')
    plt.tight_layout()
    plt.show()

elif hasattr(best_model, 'coef_'):
    print("\nCoeficientes del Modelo:")
    # Para modelos lineales multiclase
    if len(best_model.coef_.shape) > 1:
        coef_df = pd.DataFrame(best_model.coef_.T, 
                              index=feature_columns, 
                              columns=class_names)
        display(coef_df)
    else:
        coef_df = pd.DataFrame({
            'Feature': feature_columns,
            'Coefficient': best_model.coef_[0]
        }).sort_values('Coefficient', key=abs, ascending=False)
        display(coef_df)

In [ ]:
# Resumen final del proyecto
print("=== RESUMEN FINAL DEL PROYECTO ===")
print("\n1. EXTRACCIÓN DE DATOS:")
print(f"   - Dataset inicial: {df_original.shape[0]} filas, {df_original.shape[1]} columnas")
print(f"   - Dataset limpio: {df_clean.shape[0]} filas, {df_clean.shape[1]} columnas")

print("\n2. LIMPIEZA DE DATOS:")
print(f"   - Valores nulos tratados: {df_original.isnull().sum().sum()}")
print(f"   - Duplicados eliminados: {df_original.shape[0] - df_clean.shape[0]}")
print(f"   - Variables categóricas codificadas: {len(categorical_features)}")

print("\n3. ANÁLISIS EXPLORATORIO:")
print(f"   - Variables numéricas analizadas: {len(numerical_cols)}")
print(f"   - Variables categóricas analizadas: {len(categorical_cols)}")
print(f"   - Correlaciones fuertes encontradas: {len(strong_correlations)}")

print("\n4. ANÁLISIS ESTADÍSTICO:")
print("   - 3 hipótesis planteadas y evaluadas")
print("   - Tests de normalidad, ANOVA y Chi-cuadrado realizados")

print("\n5. MACHINE LEARNING:")
print(f"   - Modelos entrenados: {len(models)}")
print(f"   - Mejor modelo: {best_model_name}")
print(f"   - Mejor CV Score: {results[best_model_name]['cv_mean']:.4f}")
print(f"   - Accuracy en test: {results[best_model_name]['accuracy']:.4f}")

print("\n=== PROYECTO COMPLETADO EXITOSAMENTE ===")

# Guardar resultados finales
final_results = {
    'dataset_shape': df_clean.shape,
    'best_model': best_model_name,
    'best_accuracy': results[best_model_name]['accuracy'],
    'best_cv_score': results[best_model_name]['cv_mean'],
    'all_models_performance': {name: result['cv_mean'] for name, result in results.items()}
}

print("\nResultados finales guardados en la variable 'final_results'")